In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os

data_base_path = '../scraped_data'
os.makedirs(data_base_path, exist_ok=True)

data = []
start_page = 0
end_page = 1

page_number = start_page
max_pages = end_page

prix_min = 1

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
}

equipements_possibles = [
    'ABS',
    'Airbags',
    'CD/MP3/Bluetooth',
    'Caméra de recul',
    'Climatisation',
    'ESP',
    'Jantes aluminium',
    'Limiteur de vitesse',
    'Ordinateur de bord',
    'Radar de recul',
    'Régulateur de vitesse',
    'Sièges cuir',
    'Système de navigation/GPS',
    'Toit ouvrant',
    'Verrouillage centralisé à distance',
    'Vitres électriques'
]

# while True:
while page_number <= max_pages:
   url = f"https://www.avito.ma/fr/maroc/voitures_d_occasion-à_vendre?price={prix_min}&o={page_number}"
   print(f"\nPage-{page_number}", end=' ')
   page_number += 1
   response = requests.get(url, headers=headers)
   soupe = BeautifulSoup(response.text, 'html.parser')
   cars = soupe.select('a.sc-1jge648-0.jZXrfL')
   links = [car.get('href') for car in cars]

   if (len(links) == 0):
      break

   for i, link in enumerate(links):
      # print(f"{i}", end=' ')
      reponse_voiture = requests.get(link, headers=headers)
      soup = BeautifulSoup(reponse_voiture.text, 'html.parser')
      titre = soup.find('h1').text.strip() if soup.find('h1') else ''
      localisation = soup.select_one(
          'span.sc-1x0vz2r-0').text.strip() if soup.select_one('span.sc-1x0vz2r-0') else ''
      prix = soup.select_one('p.sc-1x0vz2r-0.lnEFFR.sc-1veij0r-10.jdRkSM').text.strip(
      ) if soup.select_one('p.sc-1x0vz2r-0.lnEFFR.sc-1veij0r-10.jdRkSM') else ''

      car_details = soup.select('div.sc-19cngu6-1.doRGIC')
      car = {
          'Titre': titre,
          'Localisation': localisation,
          'Prix': prix,
          'Année-Modèle': '',
          'Boite de vitesses': '',
          'Type de carburant': '',
          'Kilométrage': '',
          'Marque': '',
          'Modèle': '',
          'Nombre de portes': '',
          'Origine': '',
          'Première main': '',
          'Puissance fiscale': '',
          'État': ''
      }

      # les détails de la voitures
      for detail in car_details:
         label = detail.select_one('span.sc-1x0vz2r-0.bXFCIH')
         value = detail.select_one('span.sc-1x0vz2r-0.fjZBup')
         if label and value:
            key = label.text.strip()
            value = value.text.strip()
            car[key] = value

      # Initialiser tous les équipements à False
      for eq in equipements_possibles:
         car[eq] = False

      # Sélectionner tous les icônes d'équipements
      equip_icons = soup.select('div.sc-19cngu6-1.doRGIC img')
      for icon in equip_icons:
         alt_text = icon.get('alt', '').strip()
         if alt_text in equipements_possibles:
            car[alt_text] = True
      data.append(car)


df = pd.DataFrame(data, columns=car.keys())

timestamp = datetime.now().strftime("%Y_%m_%d__%H_%M_%S")
df.to_csv(
    f"{data_base_path}/data_avito_{start_page}_to_{end_page}_pages_{timestamp}.csv", index=False)